In [14]:
import os
import cv2
import numpy as np
from PIL import Image
import tkinter as tk
from tkinter import simpledialog, messagebox
from tkinter import ttk

# Ensure dataset and trainer directories exist
if not os.path.exists('dataset'):
    os.makedirs('dataset')
if not os.path.exists('trainer'):
    os.makedirs('trainer')

# Load the DNN-based face detection model
modelFile = "res10_300x300_ssd_iter_140000.caffemodel"
configFile = "deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

# Dictionary to store name-to-ID mappings
names_dict = {}

# Function to capture face images and train


def capture_and_train():
    face_name = simpledialog.askstring("Input", "Enter the name:")
    if not face_name:
        messagebox.showerror("Error", "You must enter a name!")
        return

    face_id = len(names_dict) + 1
    names_dict[face_id] = face_name

    print(f"\n [INFO] Initializing face capture for {
          face_name}. Look at the camera and wait ...")

    cam = cv2.VideoCapture(0)
    cam.set(3, 640)
    cam.set(4, 480)

    count = 0
    while True:
        ret, img = cam.read()
        if not ret:
            break
        img = cv2.flip(img, 1)
        h, w = img.shape[:2]
        blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), [
                                     104, 117, 123], False, False)
        net.setInput(blob)
        detections = net.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.6:
                box = detections[0, 0, i, 3:7] * [w, h, w, h]
                (x, y, x1, y1) = box.astype("int")
                cv2.rectangle(img, (x, y), (x1, y1), (255, 0, 0), 2)
                count += 1
                gray_face = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)[y:y1, x:x1]
                cv2.imwrite(
                    f"dataset/User.{face_id}.{str(count)}.jpg", gray_face)

        if count >= 50:
            break
        k = cv2.waitKey(100) & 0xff
        if k == 27:
            break

    cam.release()
    cv2.destroyAllWindows()

    # Training
    print(f"\n [INFO] Training faces for {face_name}. Please wait ...")
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    faces, ids = getImagesAndLabels('dataset')
    recognizer.train(faces, np.array(ids))
    recognizer.write('trainer/trainer.yml')
    print(f"\n [INFO] {len(np.unique(ids))} faces trained. Exiting Program")

# Function to get images and labels


def getImagesAndLabels(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    faceSamples = []
    ids = []

    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L')  # Convert to grayscale
        img_numpy = np.array(PIL_img, 'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faceSamples.append(img_numpy)
        ids.append(id)

    return faceSamples, ids

# Function for face recognition


def recognize_faces():
    recognizer = cv2.face.LBPHFaceRecognizer_create()

    try:
        recognizer.read('trainer/trainer.yml')  # Ensure model file exists
    except:
        messagebox.showerror(
            "Error", "No trained model found! Please capture and train faces first.")
        return

    cam = cv2.VideoCapture(0)
    cam.set(3, 640)
    cam.set(4, 480)

    while True:
        ret, img = cam.read()
        if not ret:
            break
        img = cv2.flip(img, 1)
        h, w = img.shape[:2]
        blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), [
                                     104, 117, 123], False, False)
        net.setInput(blob)
        detections = net.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Only consider detections with confidence higher than 0.5
                box = detections[0, 0, i, 3:7] * [w, h, w, h]
                (x, y, x1, y1) = box.astype("int")
                cv2.rectangle(img, (x, y), (x1, y1), (0, 255, 0), 2)

                # Crop the detected face for recognition
                gray_face = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)[y:y1, x:x1]

                # Predict the face ID using LBPH recognizer
                id, pred_confidence = recognizer.predict(gray_face)

                # Set a stricter threshold for recognizing faces
                threshold = 60  # Faces with prediction confidence > 60 will be labeled "Unknown"

                if pred_confidence < threshold:
                    name = names_dict.get(id, "Unknown")
                    confidence_text = f"  {100 - round(pred_confidence)}%"
                else:
                    name = "Unknown"  # Label face as "Unknown" if confidence is too high
                    confidence_text = "  Unknown"

                # Display the name and confidence on the screen
                cv2.putText(img, str(name), (x + 5, y - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                cv2.putText(img, str(confidence_text), (x + 5, y1 + 25),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 1)

        # Show the video frame with the detections
        cv2.imshow('camera', img)

        # Check if the window was manually closed
        if cv2.getWindowProperty('camera', cv2.WND_PROP_VISIBLE) < 1:
            break

        k = cv2.waitKey(10) & 0xff
        if k == 27:  # Press 'ESC' to exit
            break

    cam.release()
    cv2.destroyAllWindows()


# Create the main application window with enhanced UI
root = tk.Tk()
root.title("Face Recognition System")
root.geometry("700x500")
root.configure(bg="#0a0a0a")  # Darker background for a sleek look

# Style for buttons
style = ttk.Style()

style.configure("TButton",
                font=("Helvetica", 16, "bold"),
                foreground="#f1c40f",
                background="#0a0a0a",
                padding=15,
                borderwidth=1,
                focusthickness=0,
                relief="flat")

style.map("TButton",
          background=[("active", "#0a0a0a"), ("!disabled", "#0a0a0a")],
          foreground=[("pressed", "#f1c40f"), ("active", "#f1c40f")])

# Title label
label = ttk.Label(root, text="Face Recognition System", font=("Helvetica", 24, "bold"),
                  foreground="#f1c40f", background="#0a0a0a")
label.pack(pady=20)

# Buttons for capturing/training and recognition
capture_button = ttk.Button(
    root, text="Capture & Train Faces", command=capture_and_train, style="TButton")
capture_button.pack(pady=30)

recognize_button = ttk.Button(
    root, text="Recognize Faces", command=recognize_faces, style="TButton")
recognize_button.pack(pady=30)

# Add some padding to separate elements
root.pack_propagate(0)
label.pack_configure(pady=30)
capture_button.pack_configure(pady=15)
recognize_button.pack_configure(pady=15)

root.mainloop()

<table>
  <thead>
    <tr>
      <th>Model</th>
      <th>Strength</th>
      <th>Weakness</th>
      <th>Best For</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Faster R-CNN</td>
      <td>High accuracy</td>
      <td>Slower, not real-time</td>
      <td>Static images, high-precision tasks</td>
    </tr>
    <tr>
      <td>YOLO</td>
      <td>Real-time performance</td>
      <td>Less precise for small objects</td>
      <td>Real-time applications (e.g., video)</td>
    </tr>
    <tr>
      <td>MTCNN</td>
      <td>Face detection accuracy, face alignment</td>
      <td>Slower, higher computation</td>
      <td>Face detection/recognition, security</td>
    </tr>
    <tr>
      <td>RetinaNet</td>
      <td>Good balance between speed/accuracy</td>
      <td>Slower than SSD, YOLO</td>
      <td>Balanced detection tasks</td>
    </tr>
    <tr>
      <td>CenterNet</td>
      <td>Anchor-free, efficient</td>
      <td>Still not as fast as YOLO</td>
      <td>Detecting small objects in real-time</td>
    </tr>
    <tr>
      <td>EfficientDet</td>
      <td>Scalable, efficient</td>
      <td>Slower than YOLO, SSD</td>
      <td>Scalable face detection on limited hardware</td>
    </tr>
    <tr>
      <td>Haar Cascades</td>
      <td>Lightweight, fast</td>
      <td>Low accuracy in challenging conditions</td>
      <td>Simple, lightweight applications</td>
    </tr>
    <tr>
      <td>DPN</td>
      <td>Multi-scale feature maps for small objects</td>
      <td>Slower than SSD</td>
      <td>Detecting small faces, detailed tasks</td>
    </tr>
    <tr>
      <td>R-FCN</td>
      <td>Efficient, convolutional</td>
      <td>Not as fast as SSD/YOLO</td>
      <td>Tasks requiring both speed and accuracy</td>
    </tr>
  </tbody>
</table>


# Model Architecture: SSD (Single Shot Multibox Detector)